In [160]:
import pandas as pd

In [161]:
df = pd.read_csv('bagis_bagisislem.csv')

In [162]:
df = df[['isim', 'email', 'telefon', 'tutar', 'tamam_mi', 'durum_kod']]

In [163]:
# Group the data by email and telefon columns
grouped = df.groupby(["email", "telefon"])

# Filter the groups where tamam_mi is 1 and sum the tutar column
aggregated = grouped.filter(lambda x: x["tamam_mi"].all() == 1).groupby(["email", "telefon"]).agg({"tutar": "sum"})

# Reset the index of the resulting data frame
aggregated = aggregated.reset_index()

In [164]:
# Group the data by email and select the first telefon number for each group
email_first_telefon = aggregated.groupby('email')['telefon'].first().reset_index()

# Merge the email_first_telefon data frame with the aggregated data frame to keep only the first telefon for each email
aggregated = pd.merge(email_first_telefon, aggregated, on=['email', 'telefon'], how='left')

# Drop the duplicated email and telefon pairs
aggregated.drop_duplicates(subset=['email', 'telefon'], inplace=True)

In [165]:
aggregated['amount'] = aggregated['tutar'] / 100

In [166]:
aggregated = aggregated.rename({'tutar':'amount_sent_to_bank'},axis=1)

In [167]:
aggregated = aggregated.sort_values('amount',ascending=False)

In [168]:
aggregated

,email,telefon,amount_sent_to_bank,amount
68,famoduncu@gmail.com,+31652756628,800000,8000.0
189,umit43@hotmail.fr,0033670206040,720000,7200.0
133,murat@ymail.com,5326075151,500000,5000.0
40,boraozer001@hotmail.fr,0033769838150,500000,5000.0
24,ardasahmar6@gmail.com,05526242004,493500,4935.0
...,...,...,...,...
52,d.dagidir@outlook.com,05365261194,2000,20.0
204,ytugba999@gmail.com,05372804707,1000,10.0
6,Yg909500@gmail.com,0545 202 05 62,500,5.0
182,tanriseven.humeyra@gmail.com,05455861181,500,5.0


# Telefon Format

In [169]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x.replace(' ', ''))

In [170]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x.replace('(', ''))

In [171]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x.replace(')', ''))

In [172]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x.replace('O', '0'))

In [173]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x.replace('-', ''))

In [174]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: x[1:] if x.startswith('0') else x)

In [175]:
aggregated['telefon'] = aggregated['telefon'].apply(lambda x: '+90' + x if x.startswith('5') else x)

In [177]:
aggregated

,email,telefon,amount_sent_to_bank,amount
68,famoduncu@gmail.com,+31652756628,800000,8000.0
189,umit43@hotmail.fr,033670206040,720000,7200.0
133,murat@ymail.com,+905326075151,500000,5000.0
40,boraozer001@hotmail.fr,033769838150,500000,5000.0
24,ardasahmar6@gmail.com,+905526242004,493500,4935.0
...,...,...,...,...
52,d.dagidir@outlook.com,+905365261194,2000,20.0
204,ytugba999@gmail.com,+905372804707,1000,10.0
6,Yg909500@gmail.com,+905452020562,500,5.0
182,tanriseven.humeyra@gmail.com,+905455861181,500,5.0


In [128]:
aggregated[~aggregated['telefon'].str.match(r'^\+\d{1,3}\d{1,15}$')]

,email,telefon,amount_sent_to_bank,amount
189,umit43@hotmail.fr,033670206040,720000,7200.0
40,boraozer001@hotmail.fr,033769838150,500000,5000.0
155,r.aziye70@live.nl,031681468964,360000,3600.0
51,cuneyt@mega.com.tr,2124121700,200000,2000.0
125,mhmmdslm001@yahoo.de,04917661934749,50000,500.0
200,yitik.osman@gmail.com,041763408381,45000,450.0


In [129]:
aggregated[aggregated['telefon'].str.match(r'^\+\d{1,3}\d{1,15}$')]

,email,telefon,amount_sent_to_bank,amount
68,famoduncu@gmail.com,+31652756628,800000,8000.0
133,murat@ymail.com,+905326075151,500000,5000.0
24,ardasahmar6@gmail.com,+905526242004,493500,4935.0
85,hamido__35@hotmail.com,+31618031608,360000,3600.0
177,suly1983@yahoo.fr,+33472333804,250000,2500.0
...,...,...,...,...
52,d.dagidir@outlook.com,+905365261194,2000,20.0
204,ytugba999@gmail.com,+905372804707,1000,10.0
6,Yg909500@gmail.com,+905452020562,500,5.0
182,tanriseven.humeyra@gmail.com,+905455861181,500,5.0


In [131]:
aggregated.to_csv('aggregated.csv')

# Prod Loading

```python
import pandas as pd
from apps.donor.models import DonationItem
from apps.payment.models import DonationTransaction, Donation
from django.contrib.auth import get_user_model
import uuid

User = get_user_model()

df = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vRdkoW7BTw82QwMlvtb6QO5tlr8S8DokIcsAgAtXuCTwo7jzMZfZNlD6UVNJwGEt1PM5NVdwexNVHl9/pub?gid=1902504576&single=true&output=csv"
)

df["telefon"] = df["telefon"].apply(
    lambda x: "+" + str(x) if str(x).startswith("9") else x
)

old_donation = DonationItem.objects.filter(name="Eski Platform Bağışı").first()

for i, j in df.iterrows():
    try:
        new_user = User.objects.create(username=j[0], email=j[0], phone_number=j[1])
        new_user.set_password(str(uuid.uuid4()))
        new_user.save()
        new_transaction = DonationTransaction(
            first_name="",
            last_name="",
            email=new_user.email,
            phone_number=new_user.phone_number,
            amount=j[3],
            amount_sent_to_bank=j[2],
            merchant_order_id="eski-platform-bagisi",
            message="Eski Platform Bağışı",
            group_name="",
            organization_name="",
            is_complete=True,
            status_code="00",
            status_code_description="Eski Platform Bağışı (Başarılı).",
            user=new_user,
            md_code="eski-platform-bagisi",
        )
        new_transaction.save()
        new_donation = Donation.objects.create(
            donation_item=old_donation,
            amount=j[3],
            donation_transaction=new_transaction,
            user=new_transaction.user,
        )
        new_donation.save()
    except Exception as e:
        print(e)
        print(f"Error occured: {j[0]} {j[1]}")

```